In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import pickle
import time
from sklearn.model_selection import train_test_split

In [ ]:
# ============================================================================
# PATH CONFIGURATION
# ============================================================================
CSV_PATH = '../data/features/captions_preprocessed.csv'
IMAGE_FEATURES_DIR = '../data/Images'
TOKENIZER_PATH = '../data/features/tokenizer.pkl'
CHECKPOINT_PATH = "../checkpoints"

In [ ]:
# ============================================================================
# LOAD TOKENIZER & DATASET
# ============================================================================
# Memuat data tokenizer
with open(TOKENIZER_PATH, 'rb') as f:
    tokenizer_data = pickle.load(f)

word_to_idx = tokenizer_data['word_to_idx']
vocab_size = tokenizer_data['vocab_size']
max_length = tokenizer_data['max_length']

# Memuat dataset
df = pd.read_csv(CSV_PATH)
train_df, val_df = train_test_split(df, test_size=0.1, random_state=42)

# Hyperparameters
BATCH_SIZE = 64
BUFFER_SIZE = 1000
EMBEDDING_DIM = 256
UNITS = 512

In [ ]:
# ============================================================================
# DATA GENERATOR
# ============================================================================
def load_data(img_name, caption):
    # Menyesuaikan penamaan file .npy
    path_npy = os.path.join(IMAGE_FEATURES_DIR, img_name.decode('utf-8') + '.npy')
    img_tensor = np.load(path_npy)

    # Tokenisasi caption
    cap_seq = [word_to_idx.get(word, word_to_idx['<UNK>']) for word in caption.decode('utf-8').split()]
    cap_seq = tf.keras.preprocessing.sequence.pad_sequences([cap_seq], maxlen=max_length, padding='post')[0]

    return img_tensor, cap_seq

def map_func(img_name, cap):
    img_tensor, cap_seq = tf.numpy_function(load_data, [img_name, cap], [tf.float32, tf.int32])
    img_tensor.set_shape((64, 2048)) # Sesuai output InceptionV3 yang di-reshape
    cap_seq.set_shape((max_length,))
    return img_tensor, cap_seq

dataset = tf.data.Dataset.from_tensor_slices((train_df['image'].values, train_df['caption'].values))
dataset = dataset.map(map_func, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
# ============================================================================
# MODEL ARCHITECTURE
# ============================================================================
class CNN_Encoder(tf.keras.Model):
    def __init__(self, embedding_dim):
        super(CNN_Encoder, self).__init__()
        self.fc = tf.keras.layers.Dense(embedding_dim)
        # Menangani range fitur (0-17) agar stabil
        self.ln = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        # Mencegah Dead ReLU pada fitur yang banyak nol
        self.leaky_relu = tf.keras.layers.LeakyReLU(alpha=0.1)

    def call(self, x):
        x = self.fc(x)
        x = self.ln(x)
        return self.leaky_relu(x)

class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
        hidden_with_time_axis = tf.expand_dims(hidden, 1)
        score = self.V(tf.nn.tanh(self.W1(features) + self.W2(hidden_with_time_axis)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

class RNN_Decoder(tf.keras.Model):
    def __init__(self, embedding_dim, units, vocab_size):
        super(RNN_Decoder, self).__init__()
        self.units = units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.units, return_sequences=True, return_state=True)
        self.fc1 = tf.keras.layers.Dense(self.units)
        self.fc2 = tf.keras.layers.Dense(vocab_size)
        self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
        context_vector, _ = self.attention(features, hidden)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        x = self.fc1(output)
        x = tf.reshape(x, (-1, x.shape[2]))
        return self.fc2(x), state

In [ ]:
# ============================================================================
# TRAINING SETUP
# ============================================================================
encoder = CNN_Encoder(EMBEDDING_DIM)
decoder = RNN_Decoder(EMBEDDING_DIM, UNITS, vocab_size)

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0)) # Masking padding index 0
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_sum(loss_) / (tf.reduce_sum(mask) + 1e-8)

ckpt = tf.train.Checkpoint(encoder=encoder, decoder=decoder, optimizer=optimizer)
ckpt_manager = tf.train.CheckpointManager(ckpt, CHECKPOINT_PATH, max_to_keep=5)

@tf.function
def train_step(img_tensor, target):
    loss = 0
    hidden = tf.zeros((target.shape[0], UNITS))
    dec_input = tf.expand_dims([word_to_idx['startseq']] * target.shape[0], 1)

    with tf.GradientTape() as tape:
        features = encoder(img_tensor)
        for i in range(1, target.shape[1]):
            predictions, hidden = decoder(dec_input, features, hidden)
            loss += loss_function(target[:, i], predictions)
            dec_input = tf.expand_dims(target[:, i], 1)

    total_loss = (loss / int(target.shape[1]))
    trainable_variables = encoder.trainable_variables + decoder.trainable_variables

    # Hitung gradien
    gradients = tape.gradient(loss, trainable_variables)

    # Gradient Clipping untuk mencegah NaN
    gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

    optimizer.apply_gradients(zip(gradients, trainable_variables))
    return total_loss

In [ ]:
# ============================================================================
# TRAINING MODEL
# ============================================================================
EPOCHS =50
print("Starting Training...")

for epoch in range(EPOCHS):
    start = time.time()
    total_loss = 0

    for (batch, (img_tensor, target)) in enumerate(dataset):
        batch_loss = train_step(img_tensor, target)
        total_loss += batch_loss

        if batch % 50 == 0:
            print(f'Epoch {epoch+1} Batch {batch} Loss {batch_loss.numpy():.4f}')

    ckpt_manager.save()
    print(f'Epoch {epoch+1} Final Loss {total_loss/len(dataset):.4f}')
    print(f'Time taken: {time.time()-start:.2f} sec\n')

Starting Training...
Epoch 1 Batch 0 Loss 2.2978
Epoch 1 Batch 50 Loss 1.5712
Epoch 1 Batch 100 Loss 2.6343
Epoch 1 Batch 150 Loss 1.5109
Epoch 1 Batch 200 Loss 1.6656
Epoch 1 Batch 250 Loss 1.8083
Epoch 1 Batch 300 Loss 3.4815
Epoch 1 Batch 350 Loss 1.8427
Epoch 1 Batch 400 Loss 1.4981
Epoch 1 Batch 450 Loss 1.8567
Epoch 1 Batch 500 Loss 1.9831
Epoch 1 Batch 550 Loss 2.1971
Epoch 1 Final Loss 2.1044
Time taken: 301.10 sec

Epoch 2 Batch 0 Loss 1.7112
Epoch 2 Batch 50 Loss 2.2610
Epoch 2 Batch 100 Loss 2.5404
Epoch 2 Batch 150 Loss 1.8573
Epoch 2 Batch 200 Loss 1.8133
Epoch 2 Batch 250 Loss 1.6917
Epoch 2 Batch 300 Loss 1.5860
Epoch 2 Batch 350 Loss 2.6071
Epoch 2 Batch 400 Loss 2.5245
Epoch 2 Batch 450 Loss 3.0042
Epoch 2 Batch 500 Loss 1.4400
Epoch 2 Batch 550 Loss 1.6534
Epoch 2 Final Loss 1.9949
Time taken: 347.25 sec

Epoch 3 Batch 0 Loss 1.4914
Epoch 3 Batch 50 Loss 1.7848
Epoch 3 Batch 100 Loss 1.7141
Epoch 3 Batch 150 Loss 1.7679
Epoch 3 Batch 200 Loss 1.8926
Epoch 3 Batch 250 